<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesora: María Paz Raveau Morales</em><br>

</div>

# **<center>TAREA 2 COMPARANDO TOPIC MODELING Y CLUSTERING</center>**
*03 de agosto de 2025*

**Nombre Estudiante**: Cristian Tobar Morales  
**Asignatura**: Procesamiento de Lenguaje Natural


## **DESCRIPCIÓN DE LA PROBLEMÁTICA**
 

## **OBJETIVOS**

## **DESARROLLO**

### **CARGA Y ANÁLISIS EXPLORATORIO DE LOS DATOS**

#### Librerías requeridas

In [1]:
import os
%matplotlib inline
import pandas as pd
import numpy as np
import math


import string
from nltk.stem import SnowballStemmer
from wordcloud import WordCloud
from nltk import bigrams

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
from itertools import compress

import re

# visualización 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import stanza
stanza.download('es') # Ejecutar solamente una vez
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma,depparse')

2025-08-02 17:28:33 INFO: Downloaded file to C:\Users\crist\stanza_resources\resources.json
2025-08-02 17:28:33 INFO: Downloading default packages for language: es (Spanish) ...
2025-08-02 17:28:35 INFO: File exists: C:\Users\crist\stanza_resources\es\default.zip
2025-08-02 17:28:40 INFO: Finished downloading models and saved to C:\Users\crist\stanza_resources
2025-08-02 17:28:40 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-08-02 17:28:40 INFO: Downloaded file to C:\Users\crist\stanza_resources\resources.json
2025-08-02 17:28:41 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2025-08-02 17:28:41 INFO: Using device: cpu
2025-08-02 17:28:41 INFO: Loading: tokenize
2025-08-02 17:29:35 INFO: Loading: mwt
2025-08-02 17:29:36 INFO: Loading: pos
2025-08-02 17:29:39 INFO: Loading: lemma
2025-08-02 17:29:41 INFO: Loading: depparse
2025-08-02 17:29:41 INFO: Done loading processors!


In [3]:
import nltk
nltk.download('punkt') # Ejecutar solamente una vez

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from nltk import word_tokenize
from nltk.corpus import stopwords
esp_stop = stopwords.words('spanish')
esp_stop = esp_stop + ['ser','tener','deber','haber','hacer','poder','ir']
esp_stop.remove('estado')
esp_stop.remove('estados')

In [5]:
import gensim
from gensim import corpora
from gensim import models

#### Cargar e Información inicial del dataset

In [45]:
# Leer cada línea como una entrada de texto
with open('ecqq.csv', encoding='utf-8') as f:
    lines = f.readlines() # lee el archivo completo como una lista de líneas

# Limpiar saltos de línea y espacios
lines = [line.strip() for line in lines if line.strip()]

# Crear DataFrame con una sola columna
df = pd.DataFrame(lines, columns=['texto'])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10204 entries, 0 to 10203
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   10204 non-null  object
dtypes: object(1)
memory usage: 79.8+ KB


In [34]:
# Exploración inicial
df['texto'].head().to_list()

['0\tse necesitan pensiones dignas a los adultos mayores, ya que no alcanza el costo de vida tan alto en que que vivimos',
 '1\tel acceso a la salud sea de calidad, con diagnosticos que generen confianza no debiendo ir a santiago para conseguir confianza. altos costos de los servicios.',
 '2\t"conprender que el trabajo es una ""actividad"" y no un ""lugar físico"", a partir de ello, la incorporación de la flexibilidad laboral permite generar mayor motivación.',
 'junto con ello, asimilar a trabajadores honorarios como aquellos que cuentan con cto indefinido y no como trabajadores de 2° categori"',
 '3\t"- más especialistas en los consultorios, son escasos lo que lleva a horas de espera interminables']

In [ ]:
# Quitar la fila 0
df = df[df['texto'] != 'P2_1_B']

Se seleccionan distintas filas del corpus con el objetivo de revisar la data cruda y comprender el tipo de información compartida, como paso previo al procesamiento.

In [47]:
import random

for i in range(8):
    print(f"\033[1m TEXTO {i}:\033[0m")
    print(df.loc[random.randint(0, len(df))]['texto'], end="\n")

 TEXTO 0:
3792	pensiones dignas, que nos permitan vivir dignidad, pensiones actuales no nos permiten llegar el mundo de la culturas, cine, libros , porque se nos esta vetando
 TEXTO 1:
4074	"- urgencia, no es urgencia
 TEXTO 2:
demora en examen (servicios)
 TEXTO 3:
2978	"mala reforma de pensiones no esta de acuerdo con el aumento
 TEXTO 4:
956	"-salud pública no permite que las personas puedan atenderse oportunamente y salud digna.
 TEXTO 5:
1314	"educación digna y de calidad que sea transversal
 TEXTO 6:
1110	"que se potencia a carabineros y se renueve completamente.
 TEXTO 7:
- Mecanismo de rentabilidad estable


En las filas seleccionadas de forma aleatoria se observa que los ciudadanos realizan solicitudes al Estado sobre su calidad de vida, ya sea para llevar una existencia tranquila y digna o para acceder a una pensión adecuada.

### **PRE-PROCESAMIENTO**

In [50]:
# Crear una nueva columna
columna = "texto_modificado"
df[columna] = df['texto']

In [51]:
# Saber que entradas no aportan información
df[columna].value_counts().head(20)

texto_modificado
"                                               109
falta de especialistas                            8
-falta de especialistas                           6
especialistas                                     5
- falta de especialistas                          5
falta de especialistas.                           5
- mas especialistas                               4
- falta de insumos                                3
lista de espera                                   3
listas de espera                                  3
-falta de medicamentos                            3
- falta de especialista                           3
-"                                                3
- lista de espera                                 3
- calidad                                         3
menos horas de espera                             2
mejor atención                                    2
- educación gratuita y de calidad para todos      2
mala atención.                                 

#### Tokenizar

#### Lemmatizar

#### Transformar el texto en minúsculas, eliminar puntuaciones, números, espacios en blanco y salto de líneas

#### Stopwords